In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import glob
import os
from multiprocessing import Process
import sys
sys.path.append('..')
from modules import utils, constants

In [3]:
SEED = constants.SEED
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.set_random_seed(SEED)
tf.compat.v1.set_random_seed(SEED)
SEED

126

In [4]:
def run_dqn_model(model_type, steps):
    dir_name = f'seed_{SEED}_{steps}'
    parent_dir = f'../models/logs/{model_type}/biopsy_without_cost'
    path = os.path.join(parent_dir, dir_name)
    os.mkdir(path)
  
    if model_type=='dqn':
        model = utils.stable_vanilla_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dqn', filename=f'dqn_{steps}')
    elif model_type=='ddqn':
        model = utils.stable_double_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='ddqn', filename=f'ddqn_{steps}')
    elif model_type== 'dueling_dqn':
        model = utils.stable_dueling_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_dqn', filename=f'dueling_dqn_{steps}')
    elif model_type == 'dueling_ddqn':
        model = utils.stable_dueling_ddqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_ddqn', filename=f'dueling_ddqn_{steps}')
    elif model_type =='dqn_per':
        model = utils.stable_vanilla_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dqn_per', filename=f'dqn_per_{steps}', per=True)
    elif model_type == 'ddqn_per':
        model = utils.stable_double_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='ddqn_per', filename=f'ddqn_per_{steps}', per=True)
    elif model_type == 'dueling_dqn_per':
        model = utils.stable_dueling_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_dqn_per', filename=f'dueling_dqn_pre_{steps}', per=True)
    elif model_type == 'dueling_ddqn_per':
        model = utils.stable_dueling_ddqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_ddqn_per', filename=f'dueling_ddqn_per_{steps}', per=True)
    else:
        raise ValueError(f'Unknown model type - {model_type}!')
    return model

In [5]:
train_df = pd.read_csv('../new_data/train_set_basic.csv')
train_df = train_df.fillna(-1)
train_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,biopsy_proven_lupus_nephritis,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
1,1,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,1,0,...,1,3,0,0,0,0,0,0,0,1


In [6]:
train_df.iloc[1]

ana                              1
fever                            0
leukopenia                       1
thrombocytopenia                 0
auto_immune_hemolysis            0
delirium                         0
psychosis                        0
seizure                          0
non_scarring_alopecia            1
oral_ulcers                      0
cutaneous_lupus                  3
pleural_effusion                 0
pericardial_effusion             1
acute_pericarditis               0
joint_involvement                0
proteinuria                      0
biopsy_proven_lupus_nephritis    0
anti_cardioliphin_antibodies     0
anti_β2gp1_antibodies            1
lupus_anti_coagulant             0
low_c3                           0
low_c4                           0
anti_dsdna_antibody              1
anti_smith_antibody              0
label                            1
Name: 1, dtype: int64

In [6]:
train_df.label.value_counts()

0    25240
1    25160
Name: label, dtype: int64

In [7]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

((50400, 24), (50400,))

In [8]:
# model_names = ['dqn', 'ddqn', 'dueling_dqn', 'dueling_ddqn', 'dqn_per', 'ddqn_per', 'dueling_dqn_per', 
#                'dueling_ddqn_per']
model_names = ['dqn_per', 'ddqn_per', 'dueling_dqn_per', 'dueling_ddqn_per']
procs = []
steps = int(10e7)

In [9]:
# run_dqn_model(model_names[0],steps)

In [10]:
for name in model_names:
#     run_dqn_model(name, steps)
    proc = Process(target=run_dqn_model, args=(name, steps))
    procs.append(proc)
    proc.start()

The environment seed is [126]
The environment seed is [126]
The environment seed is [126]
The environment seed is [126]
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
--------------------------------------
| % time spent exploring  | 95       |
| episodes                | 100000   |
| mean 100 episode reward | -0.8     |


--------------------------------------
| % time spent exploring  | 75       |
| episodes                | 500000   |
| mean 100 episode reward | -0.9     |
| steps                   | 2582696  |
| success rate            | 0.14     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 75       |
| episodes                | 500000   |
| mean 100 episode reward | -0.7     |
| steps                   | 2583000  |
| success rate            | 0.21     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 75       |
| episodes                | 500000   |
| mean 100 episode reward | -0.8     |
| steps                   | 2578621  |
| success rate            | 0.17     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 75       |
| episodes                | 500000   |
| mean 100 episode reward | -0.8     |
| steps                  

--------------------------------------
| % time spent exploring  | 27       |
| episodes                | 1200000  |
| mean 100 episode reward | -1.1     |
| steps                   | 7644792  |
| success rate            | 0.13     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 27       |
| episodes                | 1300000  |
| mean 100 episode reward | -0.2     |
| steps                   | 7653766  |
| success rate            | 0.46     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 16       |
| episodes                | 1300000  |
| mean 100 episode reward | -1.2     |
| steps                   | 8801238  |
| success rate            | 0.13     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 16       |
| episodes                | 1300000  |
| mean 100 episode reward | -1.2     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2000000  |
| mean 100 episode reward | 0.2      |
| steps                   | 13231592 |
| success rate            | 0.74     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2900000  |
| mean 100 episode reward | -1       |
| steps                   | 14865438 |
| success rate            | 0        |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1600000  |
| mean 100 episode reward | -0.5     |
| steps                   | 13816990 |
| success rate            | 0.53     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3000000  |
| mean 100 episode reward | -0.8     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2500000  |
| mean 100 episode reward | 0.4      |
| steps                   | 17723779 |
| success rate            | 0.81     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4900000  |
| mean 100 episode reward | -0.9     |
| steps                   | 19702656 |
| success rate            | 0.06     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 5000000  |
| mean 100 episode reward | -0.8     |
| steps                   | 19949532 |
| success rate            | 0.11     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 5100000  |
| mean 100 episode reward | -0.8     |
| steps                  

...

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 21300000 |
| mean 100 episode reward | 0.4      |
| steps                   | 78190822 |
| success rate            | 0.78     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 9000000  |
| mean 100 episode reward | 0.4      |
| steps                   | 70816053 |
| success rate            | 0.8      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6400000  |
| mean 100 episode reward | 0.2      |
| steps                   | 80004501 |
| success rate            | 0.8      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 21400000 |
| mean 100 episode reward | 0.6      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 22500000 |
| mean 100 episode reward | 0.5      |
| steps                   | 84555940 |
| success rate            | 0.84     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 9900000  |
| mean 100 episode reward | 0.6      |
| steps                   | 76457224 |
| success rate            | 0.86     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6000000  |
| mean 100 episode reward | -0.1     |
| steps                   | 78528070 |
| success rate            | 0.67     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 22600000 |
| mean 100 episode reward | 0.6      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 10800000 |
| mean 100 episode reward | 0.5      |
| steps                   | 81940271 |
| success rate            | 0.84     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 23700000 |
| mean 100 episode reward | 0.4      |
| steps                   | 90911897 |
| success rate            | 0.79     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 10900000 |
| mean 100 episode reward | 0.5      |
| steps                   | 82527806 |
| success rate            | 0.84     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 7500000  |
| mean 100 episode reward | 0.3      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6900000  |
| mean 100 episode reward | 0.3      |
| steps                   | 89876854 |
| success rate            | 0.79     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 11800000 |
| mean 100 episode reward | 0.6      |
| steps                   | 87792876 |
| success rate            | 0.89     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 24900000 |
| mean 100 episode reward | 0.5      |
| steps                   | 97189675 |
| success rate            | 0.82     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 8000000  |
| mean 100 episode reward | 0.3      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 13400000 |
| mean 100 episode reward | 0.5      |
| steps                   | 96784181 |
| success rate            | 0.82     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 13500000 |
| mean 100 episode reward | 0.5      |
| steps                   | 97321838 |
| success rate            | 0.83     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 7700000  |
| mean 100 episode reward | -0       |
| steps                   | 99755430 |
| success rate            | 0.71     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 13600000 |
| mean 100 episode reward | 0.6      |
| steps                  

In [12]:
for proc in procs:
    proc.join()
print('All jobs completed and terminated successfully')

All jobs completed and terminated successfully
